## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-18-21-52-02 +0000,nyt,Trump Berates Reporter Who Asked About Journal...,https://www.nytimes.com/2025/11/18/business/me...
1,2025-11-18-21-50-51 +0000,nypost,California judge frees murder suspect without ...,https://nypost.com/2025/11/18/us-news/judge-fr...
2,2025-11-18-21-47-43 +0000,nyt,House Condemns García’s Election Scheme Amid D...,https://www.nytimes.com/2025/11/18/us/politics...
3,2025-11-18-21-45-01 +0000,nypost,GOP contender slams Long Island Rep. Suozzi as...,https://nypost.com/2025/11/18/us-news/gop-cont...
4,2025-11-18-21-43-32 +0000,bbc,UK government will not financially support Mos...,https://www.bbc.com/news/articles/cy0kxq0zp47o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2571/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,59
91,epstein,20
18,house,15
47,saudi,15
61,new,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
157,2025-11-18-15-49-00 +0000,wsj,"Once one of Trump’s most strident MAGA allies,...",https://www.wsj.com/politics/policy/how-marjor...,129
195,2025-11-18-13-24-46 +0000,cbc,Saudi crown prince MBS unaware of Khashoggi ki...,https://www.cbc.ca/news/world/washington-saudi...,129
294,2025-11-18-02-00-00 +0000,wsj,President Trump’s grip on the GOP is showing s...,https://www.wsj.com/politics/policy/trumps-gri...,126
90,2025-11-18-18-46-44 +0000,nypost,Trump’s border czar Tom Homan warns NY will se...,https://nypost.com/2025/11/18/us-news/trumps-b...,115
55,2025-11-18-20-13-26 +0000,nypost,Trump claims Saudi Arabian crown prince ‘knew ...,https://nypost.com/2025/11/18/us-news/trump-cl...,113


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
157,129,2025-11-18-15-49-00 +0000,wsj,"Once one of Trump’s most strident MAGA allies,...",https://www.wsj.com/politics/policy/how-marjor...
195,70,2025-11-18-13-24-46 +0000,cbc,Saudi crown prince MBS unaware of Khashoggi ki...,https://www.cbc.ca/news/world/washington-saudi...
90,56,2025-11-18-18-46-44 +0000,nypost,Trump’s border czar Tom Homan warns NY will se...,https://nypost.com/2025/11/18/us-news/trumps-b...
7,51,2025-11-18-21-39-00 +0000,wsj,A federal court has blocked the use of a new T...,https://www.wsj.com/politics/policy/texas-new-...
28,45,2025-11-18-21-04-00 +0000,wsj,The House passed legislation Tuesday mandating...,https://www.wsj.com/politics/policy/fight-over...
258,42,2025-11-18-09-18-41 +0000,wapo,U.N. Security Council approves Trump’s 20-poin...,https://www.washingtonpost.com/national-securi...
8,37,2025-11-18-21-36-03 +0000,nypost,Alleged China mole Linda Sun’s lavish life in ...,https://nypost.com/2025/11/18/us-news/alleged-...
220,36,2025-11-18-11-33-38 +0000,bbc,Ex-Harvard president Larry Summers steps back ...,https://www.bbc.com/news/articles/cddr199d3z0o...
33,35,2025-11-18-20-57-39 +0000,nypost,Paramount denies report it’s working with Saud...,https://nypost.com/2025/11/18/media/paramount-...
53,32,2025-11-18-20-14-16 +0000,nypost,House votes to denounce Rep. Chuy Garcia for ‘...,https://nypost.com/2025/11/18/us-news/house-vo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
